In [1]:
import cv2
import mediapipe as mp
import time
import numpy as np
import HandTrackingModule as htm
import math
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

In [2]:
wCam, hCam = 640, 480

In [3]:
cap = cv2.VideoCapture(1)
cap.set(3, wCam)
cap.set(4, hCam)
pTime = 0

detector = htm.handDetector(detectionCon=0.7)
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))
#volume.GetMute()
#volume.GetMasterVolumeLevel()
volRange = volume.GetVolumeRange()

minVol = volRange[0]
maxVol = volRange[1]
vol = 0
volBar = 400
volPer = 0

while True:
    success, img = cap.read()
    img = detector.findHands(img)
    lmList = detector.findPosition(img, draw=False)
    if len(lmList) != 0:
        #print(lmList[4], lmList[8])

        x1, y1 = lmList[4][1], lmList[4][2]
        x2, y2 = lmList[8][1], lmList[8][2]
        cx, cy = (x1 + x2)//2, (y1 + y2)//2

        cv2.circle(img, (x1, y1), 15, (255, 0, 255), cv2.FILLED)
        cv2.circle(img, (x2, y2), 15, (255, 0, 255), cv2.FILLED)
        cv2.circle(img, (cx, cy), 15, (255, 0, 255), cv2.FILLED)

        cv2.line(img, (x1, y1), (x2,y2), (255,0, 255), 3)
        length = math.hypot(x2 - x1, y2 - y1)

        ## hand range -- 40 - 290
        ## Volume range -- -30 to 0

        vol = np.interp(length, [40, 290], [minVol, maxVol])
        volBar = np.interp(length, [40, 290], [400, 150])
        volPer = np.interp(length, [40, 290], [0, 100])
        print(int(length), vol)
        if length < 50:
            cv2.circle(img, (cx, cy), 15, (0, 255, 0), cv2.FILLED)
        
        volume.SetMasterVolumeLevel(vol, None)

    cv2.rectangle(img, (50, 150), (85, 400), (0, 255, 0), 3)
    cv2.rectangle(img, (50, int(volBar)), (85, 400), (0, 255, 0), cv2.FILLED)
    cv2.putText(img, f'{int(volPer)}%', (40, 450), cv2.FONT_HERSHEY_PLAIN, 1,(0, 255, 0), 2)
    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime

    cv2.putText(img, f'FPS: {int(fps)}', (40,50), cv2.FONT_HERSHEY_PLAIN, 1,(255, 0, 0), 2)
    cv2.imshow("Image", img)
    cv2.waitKey(1)

147 -36.28745815441503
146 -36.50126708295882
149 -35.61337739036486
170 -30.43519971127686
183 -26.928831771503937
207 -20.83715944782977
223 -16.851947437005023
229 -15.489008947758165
230 -15.17157051176703
229 -15.392045685471338
228 -15.671738808617484
234 -14.201752027157752
231 -14.758281722856339
236 -13.657371890891305
237 -13.348503616640379
236 -13.680493566552244
237 -13.213870032896232
227 -15.825477264872319
228 -15.666732459707013
201 -22.443874336299118
178 -28.436585710497262
176 -28.79177890756086
158 -33.46695781605975
145 -36.74601300319891
142 -37.40911857623321
130 -40.444218871145
128 -40.93340353779513
111 -45.36435439860048
92 -50.114597306480405
92 -50.26302882850004
79 -53.41315669048629
71 -55.49589187435838
67 -56.44412058592417
57 -59.17977210124095
54 -59.88532918723645
50 -60.708490435474324
56 -59.34331979822138
51 -60.59689087544623
50 -60.92955648848007
49 -61.15195059171894
45 -62.06468439411846
45 -62.18493328995426
43 -62.52737964358795
44 -62.3777